In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#PHASE II

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import plot_model
from scipy.stats import pearsonr, spearmanr, kendalltau

In [ ]:
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
CHANNELS = 3
ssim_scores = pd.read_csv('/content/drive/MyDrive/metrics_scores_phase2.csv')
ssim_scores = ssim_scores[['Image', 'SSIM_Score']]
def load_dataset(base_path, ssim_scores):
    images = []
    labels = []
    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if file.endswith('.bmp'):
                    file_path = os.path.join(folder_path, file)
                    img = load_img(file_path, target_size=(IMAGE_HEIGHT, IMAGE_WIDTH))
                    img_array = img_to_array(img)
                    images.append(img_array)
                    if folder == 'Reference':
                        labels.append(1.0)
                    else:
                        label = ssim_scores[ssim_scores['Image'] == file]['SSIM_Score'].values[0]
                        labels.append(label)
    return np.array(images), np.array(labels)

In [ ]:
base_path = '/content/drive/MyDrive/Phase2/Stimuli/separate'
images, labels = load_dataset(base_path, ssim_scores)
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
def split_images(images):
    left_images = images[:, :, :IMAGE_WIDTH//2, :]
    right_images = images[:, :, IMAGE_WIDTH//2:, :]
    return left_images, right_images
train_left, train_right = split_images(train_images)
test_left, test_right = split_images(test_images)

In [ ]:
def build_feature_extractor(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)

    x = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.1)(x)

    x = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.4)(x)

    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu', kernel_initializer='he_uniform')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    return tf.keras.Model(inputs=input_layer, outputs=x)


In [ ]:
left_feature_extractor = build_feature_extractor(input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH // 2, CHANNELS))
right_feature_extractor = build_feature_extractor(input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH // 2, CHANNELS))

In [ ]:
def build_combined_model(left_feature_extractor, right_feature_extractor, input_shape):
    left_input = tf.keras.Input(shape=input_shape)
    right_input = tf.keras.Input(shape=input_shape)

    left_features = left_feature_extractor(left_input)
    right_features = right_feature_extractor(right_input)

    combined_features = layers.concatenate([left_features, right_features])
    combined_features = layers.Dense(512, activation='relu')(combined_features)
    combined_features = layers.BatchNormalization()(combined_features)
    combined_features = layers.Dropout(0.5)(combined_features)

    combined_features = layers.Dense(128, activation='relu')(combined_features)
    combined_features = layers.BatchNormalization()(combined_features)
    combined_features = layers.Dropout(0.5)(combined_features)

    output = layers.Dense(1)(combined_features)

    model = tf.keras.Model(inputs=[left_input, right_input], outputs=output)
    return model

In [ ]:
combined_model = build_combined_model(left_feature_extractor, right_feature_extractor, (IMAGE_HEIGHT, IMAGE_WIDTH // 2, CHANNELS))
combined_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
combined_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 128, 64, 3)]         0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 128, 64, 3)]         0         []                            
                                                                                                  
 model (Functional)          (None, 128)                  1012896   ['input_3[0][0]']             
                                                                                                  
 model_1 (Functional)        (None, 128)                  1012896   ['input_4[0][0]']             
                                                                                            

In [ ]:
def regression_accuracy(y_true, y_pred):
    return tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)

def plcc(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    return tf.py_function(func=lambda y_true, y_pred: np.float32(pearsonr(y_true, y_pred)[0]), inp=[y_true, y_pred], Tout=tf.float32)

def srocc(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    return tf.py_function(func=lambda y_true, y_pred: np.float32(spearmanr(y_true, y_pred)[0]), inp=[y_true, y_pred], Tout=tf.float32)

def rmse(y_true, y_pred):
    return tf.sqrt(tf.keras.losses.mean_squared_error(y_true, y_pred))

def krocc(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    return tf.py_function(func=lambda y_true, y_pred: np.float32(kendalltau(y_true, y_pred)[0]), inp=[y_true, y_pred], Tout=tf.float32)

In [ ]:
# Train the model
history = combined_model.fit(
    [train_left, train_right],
    train_labels,
    epochs=30,
    batch_size=32,
    validation_data=([test_left, test_right], test_labels)
)

Epoch 1/30
10/10 [==============================] - 57s 5s/step - loss: 0.1629 - mae: 0.3069 - val_loss: 0.0288 - val_mae: 0.1410
Epoch 2/30
10/10 [==============================] - 54s 6s/step - loss: 0.1510 - mae: 0.2939 - val_loss: 0.0288 - val_mae: 0.1415
Epoch 3/30
10/10 [==============================] - 53s 5s/step - loss: 0.1334 - mae: 0.2931 - val_loss: 0.0293 - val_mae: 0.1430
Epoch 4/30
10/10 [==============================] - 54s 5s/step - loss: 0.1598 - mae: 0.2892 - val_loss: 0.0254 - val_mae: 0.1328
Epoch 5/30
10/10 [==============================] - 50s 5s/step - loss: 0.1294 - mae: 0.2775 - val_loss: 0.0241 - val_mae: 0.1286
Epoch 6/30
10/10 [==============================] - 50s 5s/step - loss: 0.1359 - mae: 0.2904 - val_loss: 0.0249 - val_mae: 0.1308
Epoch 7/30
10/10 [==============================] - 50s 5s/step - loss: 0.1261 - mae: 0.2777 - val_loss: 0.0236 - val_mae: 0.1262
Epoch 8/30
10/10 [==============================] - 50s 5s/step - loss: 0.1141 - mae: 0.25

In [ ]:
metrics = combined_model.evaluate([test_left, test_right], test_labels, verbose=1)
print("\nEvaluation Metrics:")
print(f"MSE: {metrics[0]}")
print(f"MAE: {metrics[1]}")

3/3 [==============================] - 4s 1s/step - loss: 0.0187 - mae: 0.1123

Evaluation Metrics:
MSE: 0.018688136711716652
MAE: 0.11228048801422119


In [ ]:
combined_model.compile(optimizer='adam', loss='mse', metrics=[regression_accuracy, plcc, srocc, rmse, krocc])
metrics = combined_model.evaluate([test_left, test_right], test_labels, verbose=1)
print("\nEvaluation Metrics:")
print(f"MSE: {metrics[0]}")
print(f"Reg. Accuracy: {metrics[1]}")
print(f"PLCC: {metrics[2]}")
print(f"SROCC: {metrics[3]}")
print(f"RMSE: {metrics[4]}")
print(f"KROCC: {metrics[5]}")


3/3 [==============================] - 9s 1s/step - loss: 0.0187 - regression_accuracy: 15.1799 - plcc: 0.4710 - srocc: 0.4522 - rmse: 0.1123 - krocc: 0.3125

Evaluation Metrics:
MSE: 0.018688136711716652
Reg. Accuracy: 15.179885864257812
PLCC: 0.4709664285182953
SROCC: 0.4522044360637665
RMSE: 0.11228048801422119
KROCC: 0.31247952580451965


In [ ]:
distortion_types = ['Fast_Fading', 'Gaussian_Blur', 'JPEG', 'JPEG_2000', 'White_Noise']
metrics_per_distortion = {distortion_type: {} for distortion_type in distortion_types}

for distortion_type in distortion_types:
    distortion_folder = os.path.join(base_path, distortion_type)
    images = []
    labels = []
    for file in os.listdir(distortion_folder):
        if file.endswith('.bmp'):
            file_path = os.path.join(distortion_folder, file)
            img = load_img(file_path, target_size=(IMAGE_HEIGHT, IMAGE_WIDTH))
            img_array = img_to_array(img)
            images.append(img_array)

            if distortion_type == 'Reference':
                labels.append(1.0)
            else:
                label = ssim_scores[ssim_scores['Image'] == file]['SSIM_Score'].values[0]
                labels.append(label)

    images = np.array(images)
    labels = np.array(labels)

    left_images, right_images = split_images(images)

    subset_metrics = combined_model.evaluate([left_images, right_images], labels, verbose=0)

    metrics_per_distortion[distortion_type]['MSE'] = subset_metrics[0]
    metrics_per_distortion[distortion_type]['PLCC'] = subset_metrics[2]
    metrics_per_distortion[distortion_type]['SROCC'] = subset_metrics[3]
    metrics_per_distortion[distortion_type]['RMSE'] = subset_metrics[4]
    metrics_per_distortion[distortion_type]['KROCC'] = subset_metrics[5]
for distortion_type, metrics in metrics_per_distortion.items():
    print(f"\nMetrics for {distortion_type}:")
    print(f"MSE: {metrics['MSE']}")
    print(f"PLCC: {metrics['PLCC']}")
    print(f"SROCC: {metrics['SROCC']}")
    print(f"RMSE: {metrics['RMSE']}")
    print(f"KROCC: {metrics['KROCC']}")
metrics = combined_model.evaluate([test_left, test_right], test_labels, verbose=1)




Metrics for Fast_Fading:
MSE: 0.012053858488798141
PLCC: 0.7916707396507263
SROCC: 0.8249080777168274
RMSE: 0.08888654410839081
KROCC: 0.6993088126182556

Metrics for Gaussian_Blur:
MSE: 0.01767377182841301
PLCC: 0.7155044078826904
SROCC: 0.7164444327354431
RMSE: 0.112137570977211
KROCC: 0.5449308753013611

Metrics for JPEG:
MSE: 0.006941850762814283
PLCC: 0.4848615229129791
SROCC: 0.4461331367492676
RMSE: 0.06817054003477097
KROCC: 0.3408524990081787

Metrics for JPEG_2000:
MSE: 0.008008809760212898
PLCC: 0.6253728270530701
SROCC: 0.6444514393806458
RMSE: 0.07522796839475632
KROCC: 0.5213133692741394

Metrics for White_Noise:
MSE: 0.023903507739305496
PLCC: 0.7056638598442078
SROCC: 0.6147651672363281
RMSE: 0.12798111140727997
KROCC: 0.4890553057193756
3/3 [==============================] - 5s 1s/step - loss: 0.0187 - regression_accuracy: 15.1799 - plcc: 0.4710 - srocc: 0.4522 - rmse: 0.1123 - krocc: 0.3125
